# UTS

# 1.	Lakukan analisa klasifikasikan berita dengan extraksi fitur model topik modelling dengan classifier naïve bayes dan SVM 

# Preprocessing Berita

In [ ]:
import pandas as pd

# Load data from berita_cnn.csv
df = pd.read_csv('Berita.csv')

In [ ]:
# Display basic information about the dataset
print("Dataset shape:", df.shape)
df

In [ ]:
print("\nDataset info:")
df.info()

#   PREPROCESSING

In [ ]:
# Tampilkan data "isi"
df['berita']

## Hapus Missing Value dan Data Duplicat

In [ ]:
# Hapus baris dengan Missing Value di 'berita'
df.dropna(subset=['berita'], inplace=True)

# Hapus data duplikat
df.drop_duplicates(inplace=True)

## Test Cleaning

In [ ]:
import re

# Fungsi untuk membersihkan teks
def clean_text(text):
    # Pastikan input adalah string
    if not isinstance(text, str):
        return ""
        
    text = text.lower() # 1. Ubah ke huruf kecil
    
    # 2. Ganti karakter non-breaking space (U+00A0) dengan spasi biasa
    text = text.replace(u'\xa0', u' ')
    
    # 3. Hapus awalan kota dan sumber berita 
    # Pola: NAMA_KOTA,BANGSAONLINE.COM-
    text = re.sub(r'^\w+\s*,\s*bangsaonline\.com[–-]?\s*', '', text)
    
    # 4. Hapus semua karakter yang BUKAN huruf, angka, atau spasi
    text = re.sub(r'[^\w\s]', '', text)
    
    # 5. Hapus semua angka
    text = re.sub(r'\d+', '', text)
    
    # 6. Ganti spasi ganda/lebih menjadi satu spasi & hapus spasi di awal/akhir
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Terapkan pembersihan ke kolom 'isi'
df['cleaned_isi'] = df['berita'].apply(clean_text)

# Tampilkan DataFrame
display(df[['berita', 'cleaned_isi']].head())

## Tokenisasi

In [ ]:
import sys

# Perintah untuk menginstal library menggunakan path Python yang sedang aktif
!{sys.executable} -m pip install nltk

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') 

# Fungsi untuk melakukan tokenisasi
def tokenize_text(text):
    return word_tokenize(text)

# Terapkan tokenisasi ke kolom 'cleaned_isi'
df['tokenized_isi'] = df['cleaned_isi'].apply(tokenize_text)

# Tampilkan DataFrame dengan kolom hasil tokenisasi
display(df[['cleaned_isi', 'tokenized_isi']].head())

## Stopword Removal

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# Dapatkan Stop Word bahasa Indonesia
list_stopwords = set(stopwords.words('indonesian'))

# Fungsi untuk menghapus stop words
def remove_stopwords(tokens):
    return [word for word in tokens if word not in list_stopwords]

# Terapkan penghapusan Stop Word ke kolom 'tokenized_isi'
df['stopwords_removed_isi'] = df['tokenized_isi'].apply(remove_stopwords)

# Tampilkan DataFrame
display(df[['tokenized_isi', 'stopwords_removed_isi']].head())

In [ ]:
from collections import Counter

# Gabungkan semua token setelah stopword removal menjadi satu daftar
all_words_after_stopwords = [word for tokens in df['stopwords_removed_isi'] for word in tokens]

# Hitung frekuensi setiap kata
word_frequencies = Counter(all_words_after_stopwords)

# Menampilkan kata-kata yang paling umum dan frekuensinya
print("Top Most Frequent Words (Without Stemming):")
for word, frequency in word_frequencies.most_common(20): # Menampilkan 20 kata teratas
    print(f"{word}: {frequency}")

In [ ]:
# Buat DataFrame baru dengan isi berita asli, hasil preprocessing, dan kategori
processed_df = df[['berita', 'stopwords_removed_isi', 'kategori']].copy()

# Ganti nama kolom 'stopwords_removed_isi' menjadi 'hasil_preprocessing'
processed_df.rename(columns={'stopwords_removed_isi': 'hasil_preprocessing'}, inplace=True)

# Konversi frekuensi kata ke DataFrame
frequency_df = pd.DataFrame.from_dict(word_frequencies, orient='index', columns=['frequency'])
frequency_df.index.name = 'word'
frequency_df.sort_values(by='frequency', ascending=False, inplace=True)

# Simpan ke dua file CSV terpisah
processed_df.to_csv('hasil_preprocessing_beritaUTS.csv', index=False, encoding='utf-8')
frequency_df.to_csv('frekuensi_kata_beritaUTS.csv', encoding='utf-8')

print("Hasil preprocessing disimpan di 'hasil_preprocessing_beritaUTS.csv'")
print("Frekuensi kata disimpan di 'frekuensi_kata_beritaUTS.csv'")

In [ ]:
hasil_preprocessing = "hasil_preprocessing_beritaUTS.csv"  
df = pd.read_csv(hasil_preprocessing)

# Tampilkan data
df

In [ ]:
frekuensi_kata = "frekuensi_kata_beritaUTS.csv"  
df = pd.read_csv(frekuensi_kata)

# Tampilkan data
df

# Analisa klasifikasikan berita dengan extraksi fitur model topik modelling dengan classifier naïve bayes dan SVM 

In [ ]:
## SETUP

# Import library dasar
import pandas as pd
import numpy as np
import time

# Import untuk LDA & Koherensi
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore, CoherenceModel, HdpModel

# Import untuk Klasifikasi & Evaluasi
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Import untuk Visualisasi
import matplotlib.pyplot as plt


print("Semua library berhasil diimpor.")



In [ ]:
# Memuat Dataset 
try:
    df = pd.read_csv('hasil_preprocessing_beritaUTS.csv')
    print(f"\nDataset berhasil dimuat. Jumlah data: {len(df)} baris.")
    print("Contoh data awal:")
    print(df.head())
except FileNotFoundError:
    print("\nError: File tidak ditemukan. Pastikan nama file CSV sudah benar.")
    # Jika file tidak ditemukan, hentikan proses.
    # df = pd.DataFrame()

In [ ]:
## EKSTRAKSI FITUR LDA & PENCARIAN TOPIK OPTIMAL

# Siapkan data untuk Gensim
documents = [doc.split() for doc in df['hasil_preprocessing']]
dictionary = Dictionary(documents)

# Filter Kamus 
# Membuang kata yang terlalu jarang atau terlalu sering muncul.
# Ini meningkatkan kualitas topik secara signifikan.
dictionary.filter_extremes(no_below=15, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in documents]
print(f"\nKamus dibuat dan difilter. Jumlah kata unik: {len(dictionary)}")


# Estimasi Jumlah Topik dengan HDP 
print("\n--- Menjalankan HDP untuk estimasi jumlah topik... ---")
hdp_model = HdpModel(corpus=corpus, id2word=dictionary)
estimated_num_topics = len(hdp_model.print_topics())
print(f"✅ HDP mengestimasi ada sekitar: {estimated_num_topics} topik.")


# Mencari Jumlah Topik Terbaik dengan Plot Koherensi 
def compute_coherence_values_multicore(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    start_time = time.time()
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics,
                             random_state=42, passes=10, workers=3)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        current_coherence = coherencemodel.get_coherence()
        coherence_values.append(current_coherence)
        print(f"Selesai menghitung untuk {num_topics} topik. Skor Koherensi: {current_coherence:.4f}")
    total_time = time.time() - start_time
    print(f"\nTotal waktu pencarian koherensi: {total_time/60:.2f} menit")
    return coherence_values

# Atur rentang pencarian di sekitar hasil HDP
search_start = max(2, estimated_num_topics - 15)
search_limit = estimated_num_topics + 20
search_step = 5

print(f"\n--- Menjalankan pencarian koherensi dari {search_start} hingga {search_limit} topik... ---")
coherence_values = compute_coherence_values_multicore(dictionary=dictionary, corpus=corpus, texts=documents,
                                                    start=search_start, limit=search_limit, step=search_step)

# Tampilkan grafik
x = range(search_start, search_limit, search_step)
plt.figure(figsize=(10, 6))
plt.plot(x, coherence_values, marker='o')
plt.title("Pencarian Jumlah Topik Optimal", fontsize=16)
plt.xlabel("Jumlah Topik (num_topics)")
plt.ylabel("Skor Koherensi (c_v)")
plt.xticks(x)
plt.grid(True)
plt.show()

# Pilih jumlah topik terbaik (yang memiliki skor koherensi tertinggi)
optimal_num_topics = x[np.argmax(coherence_values)]
print(f"\n✅ Jumlah topik optimal yang ditemukan: {optimal_num_topics}")


# Latih Model LDA Final & Ekstrak Fitur ---
print("\n--- Melatih model LDA final dengan topik optimal... ---")
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=optimal_num_topics,
                         random_state=42, passes=15)

print("\n--- Topik-topik yang Ditemukan oleh Model LDA ---")
    # Tampilkan 15 kata teratas untuk setiap topik
for idx, topic in lda_model.print_topics(num_words=15):
    print(f"Topik: {idx}")
    print(f"Kata-kata: {topic}\n")

# Ekstrak fitur (distribusi topik) untuk setiap dokumen
def get_lda_features(lda_model, bow_corpus):
    features = []
    for doc_bow in bow_corpus:
        topic_distribution = lda_model.get_document_topics(doc_bow, minimum_probability=0)
        doc_features = [0.0] * optimal_num_topics
        for topic_id, prob in topic_distribution:
            doc_features[topic_id] = prob
        features.append(doc_features)
    return np.array(features)

X = get_lda_features(lda_model, corpus)
y = df['kategori'].values

print("Ekstraksi fitur LDA selesai.")
print(f"Bentuk matriks fitur (X): {X.shape}")

## Menggunakan SVM

In [ ]:
## PELATIHAN DAN EVALUASI MODEL KLASIFIKASI

# Bagi Data menjadi Latih dan Uji ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nData dibagi. Jumlah data latih: {len(X_train)}, Jumlah data uji: {len(X_test)}")


# Latih Model Klasifikasi (SVM) 
classifier = SVC(kernel='linear', random_state=42, probability=True)

print("\n--- Memulai pelatihan model SVM... ---")
classifier.fit(X_train, y_train)
print("Pelatihan selesai.")


# Evaluasi Kinerja Model
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\n--- HASIL EVALUASI AKHIR ---")
print(f"Akurasi Model: {accuracy * 100:.2f}%")
print("\nLaporan Klasifikasi (Classification Report):")
print(classification_report(y_test, y_pred))

## Menggunakan Naivi Bayes

In [ ]:
## PELATIHAN DAN EVALUASI MODEL KLASIFIKASI

# Bagi Data menjadi Latih dan Uji ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nData dibagi. Jumlah data latih: {len(X_train)}, Jumlah data uji: {len(X_test)}")

classifier_nb = MultinomialNB()
print("\nMemulai pelatihan model Naive Bayes...")
classifier_nb.fit(X_train, y_train)
print("Pelatihan selesai.")

y_pred_nb = classifier_nb.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("\n--- HASIL EVALUASI (NAIVE BAYES) ---")
print(f"Akurasi Model: {accuracy_nb * 100:.2f}%")
print("\nLaporan Klasifikasi (Classification Report):")
print(classification_report(y_test, y_pred_nb, zero_division=0))

## Kesimpulan


# 2.	Lakukan  analisa clutering dokumen pada data email

# Preprocessing Email

In [ ]:
import pandas as pd

# Load data from berita_cnn.csv
df = pd.read_csv('spam.csv', encoding='latin1')

In [ ]:
# Display basic information about the dataset
print("Dataset shape:", df.shape)
df

In [ ]:
print("\nDataset info:")
df.info()

#   PREPROCESSING

In [ ]:
# Tampilkan data "isi"
df['Text']

## Hapus Missing Value dan Data Duplicat

In [ ]:
# Hapus baris dengan Missing Value di 'berita'
df.dropna(subset=['Text'], inplace=True)

# Hapus data duplikat
df.drop_duplicates(inplace=True)

## Test Cleaning

In [ ]:
import re

# Fungsi untuk membersihkan teks
def clean_text(text):
    # Pastikan input adalah string
    if not isinstance(text, str):
        return ""
        
    text = text.lower() # 1. Ubah ke huruf kecil
    
    # 2. Ganti karakter non-breaking space (U+00A0) dengan spasi biasa
    text = text.replace(u'\xa0', u' ')
    
    # 3. Hapus awalan kota dan sumber berita 
    # Pola: NAMA_KOTA,BANGSAONLINE.COM-
    text = re.sub(r'^\w+\s*,\s*bangsaonline\.com[–-]?\s*', '', text)
    
    # 4. Hapus semua karakter yang BUKAN huruf, angka, atau spasi
    text = re.sub(r'[^\w\s]', '', text)
    
    # 5. Hapus semua angka
    text = re.sub(r'\d+', '', text)
    
    # 6. Ganti spasi ganda/lebih menjadi satu spasi & hapus spasi di awal/akhir
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Terapkan pembersihan ke kolom 'isi'
df['cleaned_isi'] = df['Text'].apply(clean_text)

# Tampilkan DataFrame
display(df[['Text', 'cleaned_isi']].head())

## Tokenisasi

In [ ]:
import sys

# Perintah untuk menginstal library menggunakan path Python yang sedang aktif
!{sys.executable} -m pip install nltk

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') 

# Fungsi untuk melakukan tokenisasi
def tokenize_text(text):
    return word_tokenize(text)

# Terapkan tokenisasi ke kolom 'cleaned_isi'
df['tokenized_isi'] = df['cleaned_isi'].apply(tokenize_text)

# Tampilkan DataFrame dengan kolom hasil tokenisasi
display(df[['cleaned_isi', 'tokenized_isi']].head())

## Stopword Removal

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# Dapatkan Stop Word bahasa Indonesia
list_stopwords = set(stopwords.words('english'))

# Fungsi untuk menghapus stop words
def remove_stopwords(tokens):
    return [word for word in tokens if word not in list_stopwords]

# Terapkan penghapusan Stop Word ke kolom 'tokenized_isi'
df['stopwords_removed_isi'] = df['tokenized_isi'].apply(remove_stopwords)

# Tampilkan DataFrame
display(df[['tokenized_isi', 'stopwords_removed_isi']].head())

In [ ]:
from collections import Counter

# Gabungkan semua token setelah stopword removal menjadi satu daftar
all_words_after_stopwords = [word for tokens in df['stopwords_removed_isi'] for word in tokens]

# Hitung frekuensi setiap kata
word_frequencies = Counter(all_words_after_stopwords)

# Menampilkan kata-kata yang paling umum dan frekuensinya
print("Top Most Frequent Words (Without Stemming):")
for word, frequency in word_frequencies.most_common(20): # Menampilkan 20 kata teratas
    print(f"{word}: {frequency}")

In [ ]:
# Buat DataFrame baru dengan isi berita asli, hasil preprocessing, dan kategori
processed_df = df[['Text', 'stopwords_removed_isi']].copy()

# Ganti nama kolom 'stopwords_removed_isi' menjadi 'hasil_preprocessing'
processed_df.rename(columns={'stopwords_removed_isi': 'hasil_preprocessing'}, inplace=True)

# Konversi frekuensi kata ke DataFrame
frequency_df = pd.DataFrame.from_dict(word_frequencies, orient='index', columns=['frequency'])
frequency_df.index.name = 'word'
frequency_df.sort_values(by='frequency', ascending=False, inplace=True)

# Simpan ke dua file CSV terpisah
processed_df.to_csv('hasil_preprocessing_emailUTS.csv', index=False, encoding='utf-8')
frequency_df.to_csv('frekuensi_kata_emailUTS.csv', encoding='utf-8')

print("Hasil preprocessing disimpan di 'hasil_preprocessing_emailUTS.csv'")
print("Frekuensi kata disimpan di 'frekuensi_kata_emailUTS.csv'")

In [ ]:
hasil_preprocessing = "hasil_preprocessing_emailUTS.csv"  
df = pd.read_csv(hasil_preprocessing)

# Tampilkan data
df

In [ ]:
frekuensi_kata = "frekuensi_kata_emailUTS.csv"  
df = pd.read_csv(frekuensi_kata)

# Tampilkan data
df

# Analisa clutering dokumen pada data email

In [ ]:
## SETUP UNTUK CLUSTERING EMAIL

import pandas as pd
import numpy as np
import time
from ast import literal_eval # PENTING: Untuk membaca kolom 'tokens' dari CSV

# Import untuk LDA & Koherensi
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore, CoherenceModel, HdpModel

# Import untuk Clustering
from sklearn.cluster import KMeans

# Import untuk Visualisasi
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score # Import Silhouette Score


print("Semua library untuk clustering berhasil diimpor.")

In [ ]:
# Memuat Dataset 
try:
    df = pd.read_csv('hasil_preprocessing_emailUTS.csv')
    print(f"\nDataset berhasil dimuat. Jumlah data: {len(df)} baris.")
    print("Contoh data awal:")
    print(df.head())
except FileNotFoundError:
    print("\nError: File tidak ditemukan. Pastikan nama file CSV sudah benar.")
    # Jika file tidak ditemukan, hentikan proses.
    # df = pd.DataFrame()

In [ ]:
df['tokens'] = df['hasil_preprocessing'].apply(literal_eval)
print(f"Data bersih dimuat. Jumlah data: {len(df)} baris.")

In [ ]:
## EKSTRAKSI FITUR LDA & PENCARIAN TOPIK OPTIMAL

# Siapkan data untuk Gensim
documents = df['tokens'].tolist()
dictionary = Dictionary(documents)
dictionary.filter_extremes(no_below=15, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in documents]
print(f"\nKamus dibuat dan difilter. Jumlah kata unik: {len(dictionary)}")


# Estimasi Jumlah Topik dengan HDP 
print("\n--- Menjalankan HDP untuk estimasi jumlah topik... ---")
hdp_model = HdpModel(corpus=corpus, id2word=dictionary)
estimated_num_topics = len(hdp_model.print_topics())
print(f"✅ HDP mengestimasi ada sekitar: {estimated_num_topics} topik.")


# Mencari Jumlah Topik Terbaik dengan Plot Koherensi 
def compute_coherence_values_multicore(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    start_time = time.time()
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics,
                             random_state=42, passes=10, workers=3)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        current_coherence = coherencemodel.get_coherence()
        coherence_values.append(current_coherence)
        print(f"Selesai menghitung untuk {num_topics} topik. Skor Koherensi: {current_coherence:.4f}")
    total_time = time.time() - start_time
    print(f"\nTotal waktu pencarian koherensi: {total_time/60:.2f} menit")
    return coherence_values

# Atur rentang pencarian di sekitar hasil HDP
search_start = max(2, estimated_num_topics - 15)
search_limit = estimated_num_topics + 20
search_step = 5

print(f"\n--- Menjalankan pencarian koherensi dari {search_start} hingga {search_limit} topik... ---")
coherence_values = compute_coherence_values_multicore(dictionary=dictionary, corpus=corpus, texts=documents,
                                                    start=search_start, limit=search_limit, step=search_step)

# Tampilkan grafik
x = range(search_start, search_limit, search_step)
plt.figure(figsize=(10, 6))
plt.plot(x, coherence_values, marker='o')
plt.title("Pencarian Jumlah Topik Optimal", fontsize=16)
plt.xlabel("Jumlah Topik (num_topics)")
plt.ylabel("Skor Koherensi (c_v)")
plt.xticks(x)
plt.grid(True)
plt.show()

# Pilih jumlah topik terbaik (yang memiliki skor koherensi tertinggi)
optimal_num_topics = x[np.argmax(coherence_values)]
print(f"\n✅ Jumlah topik optimal yang ditemukan: {optimal_num_topics}")


# Latih Model LDA Final & Ekstrak Fitur ---
print("\n--- Melatih model LDA final dengan topik optimal... ---")
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=optimal_num_topics,
                         random_state=42, passes=15)

print("\n--- Topik-topik yang Ditemukan oleh Model LDA ---")
    # Tampilkan 15 kata teratas untuk setiap topik
for idx, topic in lda_model.print_topics(num_words=15):
    print(f"Topik: {idx}")
    print(f"Kata-kata: {topic}\n")

# Ekstrak fitur (distribusi topik) untuk setiap dokumen
def get_lda_features(lda_model, bow_corpus):
    features = []
    for doc_bow in bow_corpus:
        topic_distribution = lda_model.get_document_topics(doc_bow, minimum_probability=0)
        doc_features = [0.0] * optimal_num_topics
        for topic_id, prob in topic_distribution:
            doc_features[topic_id] = prob
        features.append(doc_features)
    return np.array(features)

X = get_lda_features(lda_model, corpus)

print("Ekstraksi fitur LDA selesai.")
print(f"Bentuk matriks fitur (X): {X.shape}")

In [ ]:
# Menemukan K Optimal dengan Silhouette Score 
print("\n--- Mencari K Optimal dengan Silhouette Score (Metode Kuantitatif) ---")
K_range = range(2, 16)
silhouette_scores = []
for k in K_range:
    kmeans_model = KMeans(n_clusters=k, random_state=42, n_init='auto').fit(X)
    score = silhouette_score(X, kmeans_model.labels_)
    silhouette_scores.append(score)
    print(f"Untuk K = {k}, Silhouette Score-nya adalah {score:.4f}")

# Visualisasikan hasilnya
plt.figure(figsize=(8, 6)) # Ukuran plot disesuaikan
plt.plot(K_range, silhouette_scores, 'ro-')
plt.xlabel('Jumlah Cluster (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score untuk Setiap Nilai K')
plt.grid(True)
plt.show()

# Jalankan K-Means dengan K Terbaik 
# Pilih K yang memberikan Silhouette Score TERTINGGI secara otomatis
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\n✅ Berdasarkan Silhouette Score, K terbaik adalah: {optimal_k}")

print(f"\n--- Menjalankan K-Means final dengan K={optimal_k}... ---")
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init='auto')
# Ganti 'df' dengan nama DataFrame yang berisi data bersih Anda
df['cluster'] = kmeans.fit_predict(X)

# Analisis Hasil Cluster ---
print("\nContoh email dari setiap cluster yang ditemukan:")
for i in range(optimal_k):
    print(f"\n----- CLUSTER {i} -----")
    sample_emails = df[df['cluster'] == i]['Text'].head(2).tolist()
    for email_text in sample_emails:
        print(f"- {str(email_text)[:250]}...")

print("\n✅ Proses clustering selesai.")